In [14]:
import pandas as pd
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
import statistics
from scipy import spatial, stats
import numpy as np
from statistics import mean
import operator
from scipy.stats import ttest_ind
from sklearn.linear_model import LinearRegression
from sklearn. manifold import MDS
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.decomposition import PCA
import os

Read in the meta data an theta matrix in dataframes and reset their indices.

In [12]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

In [15]:
path = find('meta.csv','C:///')
data = pd.read_csv(path)

In [3]:
path = find('thetamatrix50.csv', 'C://')
theta = pd.read_csv(path)

In [4]:
data = data.reset_index(drop=True)

Create a dataframe that reports for every pair of targets and acquirers the median topic proportions over all their documents, if the document was before or after the effective date, the similarity between the target and acquirer vector and their industries.

In [5]:
proportions = pd.DataFrame(columns=['Target', 'Acquiror', 'TargetTopics', 'AcquirorTopics', 'Period', 'Similarity'])
nlp = English()
tokenizer = Tokenizer(nlp.vocab)
target_text_length = list()
acquirer_text_length = list()
before = list()
after = list()
valid_pair_before = list()
valid_pair_after = list()
same_industry = list()
diff_industry = list()
for company in data['companydate'].unique():
    target = list(data[data['companydate']==company]['Target.Name'])[0]
    acquirer = list(data[data['companydate']==company]['Acquiror.Name'])[0]
    target_industry = list(data[data['Target.Name']==target]['Target.Macro.Industry'])[0]
    acquiror_industry = list(data[data['Acquiror.Name']==acquirer]['Acquiror.Macro.Industry'])[0]    
    year = list(data[data['companydate']==company]['DealYear'])[0]
    docyear = list(data[data['companydate']==company]['DocumentYear'])[0]
    pair = (target, acquirer)
    period =list(data[data['companydate']==company]['period'])[0]
    target_topics= list()
    acquiror_topics = list()
    if (period == 'before') and (pair not in before):
        before.append(pair)
        indices = data[(data['Target.Name']==target)&(data['Acquiror.Name']==acquirer)&(data['period']=='before')].index
        target_indices = list()
        acquirer_indices = list()
        target_length = 0
        acquirer_length = 0
        for index in indices:
            if data.iloc[index]['companytype'] == 'Target':
                target_indices.append(index)
            if data.iloc[index]['companytype'] ==  'Acquiror':
                acquirer_indices.append(index)
        for index in target_indices:
            text = data.iloc[index]['clean_text']
            tokens = tokenizer(text)
            target_length += len(tokens)
        for index in acquirer_indices:
            text = data.iloc[index]['clean_text']
            tokens = tokenizer(text)
            acquirer_length += len(tokens)
        target_text_length.append(target_length)
        acquirer_text_length.append(acquirer_length)
        if target_length >=162 and acquirer_length >=162:
            valid_pair_before.append(pair)
            for column in theta.columns:
                if column != 'Unnamed: 0':
                    target_topics.append(theta.iloc[target_indices][column].median())
                    acquiror_topics.append(theta.iloc[acquirer_indices][column].median())
            sum_target_topics=sum(target_topics)
            target_topics = [i/sum_target_topics for i in target_topics]
            sum_acquiror_topics = sum(acquiror_topics)
            acquiror_topics = [i/sum_acquiror_topics for i in acquiror_topics]
            similarity = 1/(1+spatial.distance.euclidean(target_topics, acquiror_topics))
            if target_industry == acquiror_industry:
                same_industry.append(similarity)
            if target_industry != acquiror_industry:
                diff_industry.append(similarity)
            props = pd.DataFrame({'Target':target,'Acquiror':acquirer,'TargetTopics':[target_topics],'AcquirorTopics':[acquiror_topics],'Period':period,
                                 'Target industry':target_industry,'Acquirer industry':acquiror_industry,
                                 'Similarity':similarity,'Year':int(year), 'DocYear':int(docyear)})
            proportions = proportions.append(props)
    if (period == 'after') and (pair not in after):
        after.append(pair)
        indices = data[(data['Target.Name']==target)&(data['Acquiror.Name']==acquirer)&(data['period']=='after')].index
        target_indices = list()
        acquirer_indices = list()
        target_length = 0
        acquirer_length = 0
        for index in indices:
                if data.iloc[index]['companytype'] == 'Target':
                    target_indices.append(index)
                if data.iloc[index]['companytype'] ==  'Acquiror':
                    acquirer_indices.append(index)
        for index in target_indices:
            text = data.iloc[index]['clean_text']
            tokens = tokenizer(text)
            target_length += len(tokens)
        for index in acquirer_indices:
            text = data.iloc[index]['clean_text']
            tokens = tokenizer(text)
            acquirer_length += len(tokens)
        target_text_length.append(target_length)
        acquirer_text_length.append(acquirer_length)
        if target_length >=162 and acquirer_length >=162:
            valid_pair_after.append(pair)
            for column in theta.columns:
                if column != 'Unnamed: 0':
                    target_topics.append(theta.iloc[target_indices][column].median())
                    acquiror_topics.append(theta.iloc[acquirer_indices][column].median())
            sum_target_topics=sum(target_topics)
            target_topics = [i/sum_target_topics for i in target_topics]
            sum_acquiror_topics = sum(acquiror_topics)
            acquiror_topics = [i/sum_acquiror_topics for i in acquiror_topics]                    
            similarity = 1/(1+spatial.distance.euclidean(target_topics, acquiror_topics))
            if target_industry == acquiror_industry:
                same_industry.append(similarity)
            if target_industry != acquiror_industry:
                diff_industry.append(similarity)
            props = pd.DataFrame({'Target':target,'Acquiror':acquirer,'TargetTopics':[target_topics],'AcquirorTopics':[acquiror_topics],'Period':period,
                                 'Target industry':target_industry,'Acquirer industry':acquiror_industry,
                                  'Similarity':similarity, 'Year':int(year),'DocYear':int(docyear)})
            proportions = proportions.append(props)


C:\Users\20150624\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Add text length of targets and acquirers and compute the median length of a company's text.

In [6]:
text_length = target_text_length+acquirer_text_length

In [7]:
statistics.median(text_length)

162.0

In [8]:
proportions = proportions.reset_index(drop=True)

Compute the similarity between a target and acquirer by iterating over the rows in the dataframe. Similarity is based on the topic proportions. Also compute the similarity between a target and the mean of all other acquirers and an acquirer and the mean of all other targets. 

In [9]:
similarities = pd.DataFrame(columns=['Target', 'Acquirer', 'Period', 'Similarity', 'Mean Similarity Other Acquirers', 'Mean Similarity Other Targets'])
similarity_highest = 0
other_acquirers_higher =0
other_targets_higher = 0
for index, row in proportions.iterrows():
    other_sim_target = list()
    other_sim_acquirer = list()
    sim = row['Similarity']
    period = row['Period']
    acquirer_topics = row['AcquirorTopics']
    target_topics = row['TargetTopics']
    if period == 'before':
        for index2, row2 in proportions.iterrows():
            if (index != index2) and (row2['Period']=='before'):
                at = row2['AcquirorTopics']
                tt = row2['TargetTopics']
                simt = 1/(1+spatial.distance.euclidean(target_topics, at))
                sima = 1/(1+spatial.distance.euclidean(acquirer_topics, tt))
                other_sim_target.append(simt)
                other_sim_acquirer.append(sima)
    if period == 'after':
         for index2, row2 in proportions.iterrows():
            if (index != index2) and (row2['Period']=='after'):
                at = row2['AcquirorTopics']
                tt = row2['TargetTopics']
                simt = 1/(1+spatial.distance.euclidean(target_topics, at))
                sima = 1/(1+spatial.distance.euclidean(acquirer_topics, tt))
                other_sim_target.append(simt)
                other_sim_acquirer.append(sima)
    if (sim >= mean(other_sim_target)) and (sim >= mean(other_sim_acquirer)):
        similarity_highest +=1
    if mean(other_sim_target) > sim:
        other_acquirers_higher +=1
    if mean(other_sim_acquirer) > sim:
        other_targets_higher +=1
    if row['Target industry'] == row['Acquirer industry']:
        ind = 'Same'
    elif row['Target industry'] != row['Acquirer industry']:
        ind = 'Different'
    similarity = pd.DataFrame({'Target':row['Target'],'TargetIndustry':row['Target industry'], 'Acquirer': row['Acquiror'], 'AcquirerIndustry':row['Acquirer industry'],
                               'Period':row['Period'],'Similarity':sim,'Mean Similarity Other Acquirers':mean(other_sim_target),
                              'Mean Similarity Other Targets':mean(other_sim_acquirer), 'Industry':ind}, index=[index])
    similarities = similarities.append(similarity)


Print how often the similarity between a target and acquirer was higher than the target's similarity with the mean of other acquirers and the acquirer's similarity with the mean of other targets.

In [97]:
similarities
print(similarity_highest)
print(other_acquirers_higher)
print(other_targets_higher)

104
61
74


T-test to check if the similarity between companies from the same industry is significantly different from the similarity between companies from different industries.

In [10]:
similarities_same = similarities[similarities['Industry']=='Same']
similarities_diff = similarities[similarities['Industry']=='Different']
ttest_ind(similarities_same['Similarity'], similarities_diff['Similarity'])

Ttest_indResult(statistic=0.8196405587843935, pvalue=0.4133683464540976)

In [11]:
print(similarities_same['Similarity'].mean())
print(similarities_diff['Similarity'].mean())

0.6702331534052856
0.6586356219671027


In [100]:
similarities.to_csv('C:/similarities.csv')

T-test to check if the similarity between a target and acquirer was significantly different from the mean similarity between a target and all other acquirers. 

In [112]:
ttest_ind(similarities['Similarity'], similarities['Mean Similarity Other Acquirers'])

Ttest_indResult(statistic=4.6923917661034364, pvalue=3.675206116933575e-06)

T-test to check if the similarity between a target and acquirer was significantly different from the mean similarity between an acquirer and all other targets. 

In [111]:
ttest_ind(similarities['Similarity'], similarities['Mean Similarity Other Targets'])

Ttest_indResult(statistic=4.664803090018925, pvalue=4.176027738222112e-06)

In [109]:
print(similarities['Similarity'].mean())
print(similarities['Mean Similarity Other Acquirers'].mean())
print(similarities['Mean Similarity Other Targets'].mean())

0.664713174499612
0.6276231359470769
0.627623135947077


Subsets of data with only the data before or after the effective date. Create a dataframe with only the complete data before and after.

In [113]:
similarities_before = similarities[similarities['Period']=='before']
similarities_after = similarities[similarities['Period']=='after']

In [114]:
similarities_time = pd.DataFrame(columns=['Target', 'Acquirer', 'Before', 'After'])
for index, row in similarities_before.iterrows():
    target = row['Target']
    acquirer = row['Acquirer']
    before = row['Similarity']
    if len(similarities_after[(similarities_after['Target']==target) & (similarities_after['Acquirer']==acquirer)])==1:
        after = similarities_after[(similarities_after['Target']==target) & (similarities_after['Acquirer']==acquirer)]['Similarity'].values[0]
        similar = pd.DataFrame({'Target':target, 'Acquirer':acquirer, 'Before':before,'After':after}, index=[index])
        similarities_time = similarities_time.append(similar)

In [115]:
similarities_time

,Target,Acquirer,Before,After
1,Specialty Therapeutic Hldg Inc,Centene Corp,0.718170,0.713517
3,Integra Partners LLC,Tufts Assocd Health Plans Inc,0.757854,0.636774
4,Healthagen LLC,Aetna Inc,0.710338,0.629394
5,Aetna Inc-Missouri Medicaid,WellCare Health Plans Inc,0.756793,0.874802
6,Coventry Health Care Inc,Aetna Inc,0.635373,0.759007
8,American Health Network Inc,UnitedHealth Group Inc,0.560581,0.499416
9,AIG Sigorta AS,GIG,0.800449,0.696382
12,All Points EAP & Orgal Svcs,AllOne Health Resource Inc,0.873600,0.780150
13,Amerigroup Virginia Inc,Inova Health System,0.686782,0.589217
15,Sec Life Ins Co of America,Ameritas Life Insurance Corp,0.770861,0.559292


T-test to check if the similarity between companies before the effective date differs significantly from the effective date after.

In [116]:
ttest_ind(similarities_time['Before'], similarities_time['After'])

Ttest_indResult(statistic=0.15495107362743127, pvalue=0.8771438538903779)

In [117]:
print(similarities_time['Before'].mean())
print(similarities_time['After'].mean())

0.6672408757497253
0.6641315898060943


In [426]:
similarities_time.to_csv('C:/similaritiestime.csv')